In [33]:
# Sam Brown
# sam_brown@mines.edu
# June 12 2025
# Neural net for regression, predicting total_delta

import sys
sys.path.append("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF")

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

df = pd.read_csv("/Users/sambrown04/Documents/SURF/Preproc_data/averages_events_2011-13.csv")
df = df.iloc[1:-1] # account for shift to get mins until feature

In [11]:
# Define Features and Target
X = df[[ 'tide_height', 'tide_change', 'mins_since']]
y = df['total_delta'].values.reshape(-1,1)

# Split to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = .2, random_state = 42)

# Standardize features
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Standardize target
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

In [13]:
# Neural net
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(3,16)
        self.fc2 = nn.Linear(16,8)
        self.output = nn.Linear(8,1)

    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.output(x) # Regressing no activation fcn
        return x


In [25]:
# Instantiate
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = .01)

In [27]:
# Training loop
epochs = 300
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad() # Clears grad

    # Predictions and loss
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    #Backprop
    loss.backward()

    # Update params
    optimizer.step()
    
    if (epoch+1) % 20 == 0: # Print update to ensure no problems
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [20/300], Loss: 0.1942
Epoch [40/300], Loss: 0.1034
Epoch [60/300], Loss: 0.0968
Epoch [80/300], Loss: 0.0936
Epoch [100/300], Loss: 0.0916
Epoch [120/300], Loss: 0.0902
Epoch [140/300], Loss: 0.0893
Epoch [160/300], Loss: 0.0886
Epoch [180/300], Loss: 0.0879
Epoch [200/300], Loss: 0.0873
Epoch [220/300], Loss: 0.0869
Epoch [240/300], Loss: 0.0865
Epoch [260/300], Loss: 0.0861
Epoch [280/300], Loss: 0.0858
Epoch [300/300], Loss: 0.0855


In [29]:
# Eval
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor)
    y_pred = y_scaler.inverse_transform(y_pred_scaled.detach().numpy())
    y_test_orig = y_scaler.inverse_transform(y_test_tensor.detach().numpy())

# predictions vs actual
y_pred_flat = y_pred.flatten() # reshape to 1D
y_test_flat = y_test_orig.flatten()

for i in range(len(y_pred_flat)):
    print(f"Predicted: {y_pred_flat[i]:.2f}, Actual: {y_test_flat[i]:.2f}")


Predicted: 0.44, Actual: 0.48
Predicted: 0.28, Actual: 0.25
Predicted: 0.44, Actual: 0.48
Predicted: 0.43, Actual: 0.43
Predicted: 0.33, Actual: 0.35
Predicted: 0.37, Actual: 0.36
Predicted: 0.50, Actual: 0.49
Predicted: 0.48, Actual: 0.48
Predicted: 0.35, Actual: 0.36
Predicted: 0.49, Actual: 0.46
Predicted: 0.32, Actual: 0.31
Predicted: 0.50, Actual: 0.49
Predicted: 0.27, Actual: 0.26
Predicted: 0.33, Actual: 0.33
Predicted: 0.31, Actual: 0.33
Predicted: 0.32, Actual: 0.32
Predicted: 0.49, Actual: 0.49
Predicted: 0.37, Actual: 0.36
Predicted: 0.44, Actual: 0.44
Predicted: 0.42, Actual: 0.41
Predicted: 0.47, Actual: 0.46
Predicted: 0.49, Actual: 0.50
Predicted: 0.33, Actual: 0.35
Predicted: 0.32, Actual: 0.28
Predicted: 0.32, Actual: 0.33
Predicted: 0.47, Actual: 0.47
Predicted: 0.32, Actual: 0.34
Predicted: 0.36, Actual: 0.35
Predicted: 0.32, Actual: 0.31
Predicted: 0.30, Actual: 0.30
Predicted: 0.33, Actual: 0.32
Predicted: 0.37, Actual: 0.36
Predicted: 0.32, Actual: 0.34
Predicted:

In [31]:
r2 = r2_score(y_test_orig, y_pred)
print(r2)

0.9164748787879944
